In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import gc

In [2]:
train_X = pd.read_csv('music_data_raw/train_X.csv')
val_X = pd.read_csv('music_data_raw/valid_X.csv')
train_y = pd.read_csv('music_data_raw/train_Y.csv')
val_y = pd.read_csv('music_data_raw/valid_Y.csv')

In [3]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length']

In [5]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960524 entries, 0 to 5960523
Data columns (total 19 columns):
Unnamed: 0                int64
msno                      int64
song_id                   int64
source_screen_name        int64
source_system_tab         int64
source_type               int64
song_length               float64
genre_ids                 int64
artist_name               int64
composer                  int64
lyricist                  int64
language                  int64
city                      int64
bd                        int64
gender                    int64
registered_via            int64
registration_init_time    object
expiration_date           object
time                      float64
dtypes: float64(2), int64(15), object(2)
memory usage: 864.0+ MB


In [19]:
print(np.sum(train_X.isnull()))
print(train_X.shape)

Unnamed: 0                 0
msno                       0
song_id                    0
source_screen_name         0
source_system_tab          0
source_type                0
song_length               84
genre_ids                  0
artist_name                0
composer                   0
lyricist                   0
language                   0
city                       0
bd                         0
gender                     0
registered_via             0
registration_init_time     0
expiration_date            0
time                       0
dtype: int64
(5960524, 19)


In [20]:
all_train = pd.concat([train_X, train_y])
all_train.shape

/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(11921047, 21)

In [21]:
np.sum(train_y.isnull())

5703725          0
0.0        1533871
dtype: int64

In [23]:
train_y.head(100)

,5703725,0.0
0,8720686,NaN
1,9803651,NaN
2,6306802,1.0
3,5253583,0.0
4,7647521,NaN
5,1670641,0.0
6,1790915,0.0
7,7521379,NaN
8,278266,1.0
9,5396792,1.0


In [16]:
all_train.dropna(axis=0, how='any').shape

(0, 21)

In [7]:
# numRow = len(all_data)
# def checkCol(df, colName, numRow, isCata = True):
#     print(colName, ':')
#     print('Any NA?: ', df[colName].isnull().values.any())
#     if df[colName].isnull().values.any():
#         numNA = sum(df[colName].isnull().values)
#         print ('    # of NA: ', numNA)
#         print ('    NA%:     ', numNA/numRow)
#     if isCata:
#         levelList = df[colName].unique()
#         print('Different levels: ', len(levelList), levelList)
#         fillna(df, colName, isCata = True)
        
#     else:
#         print('range: ', min(df[colName].astype(float).dropna()), max(df[colName].astype(float).dropna()))
#         fillna(df, colName, isCata = False)
        
# def fillna(df, colName, isCata = True):
#     if isCata:
#         df[colName] = df[colName].fillna('0')
#     else:
#         if df[colName].isnull().values.any():
#             df[colName+'_dm'] = df[colName].isnull().astype(int)
#         df[colName] = df[colName].fillna(0)
#     return df

In [4]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(train_X, train_y)
print("model score: %.3f" % clf.score(val_X, val_y))
print(classification_report(val_y, clf.predict(val_X)))
print(roc_auc_score(val_y, clf.predict_proba(val_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [15]:
gc.collect()

153